In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, r2_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import pickle
import geopandas as gpd
import numpy as np
from pykrige.rk import Krige
import matplotlib.pyplot as plt

---

In [2]:
res  =pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/outputs/sample_b/regression_fine_tuning.csv', index_col=0)
for params, df in res.groupby(['param_max_depth', 'param_min_samples_leaf']):
    print(params, '\n', df[['mean_test_score', 'std_test_score']].mean(), '\n')

(6, 1) 
 mean_test_score    0.548423
std_test_score     0.018412
dtype: float64 

(6, 5) 
 mean_test_score    0.548270
std_test_score     0.018491
dtype: float64 

(6, 10) 
 mean_test_score    0.547189
std_test_score     0.018867
dtype: float64 

(50, 1) 
 mean_test_score    0.703398
std_test_score     0.020050
dtype: float64 

(50, 5) 
 mean_test_score    0.688991
std_test_score     0.020761
dtype: float64 

(50, 10) 
 mean_test_score    0.666000
std_test_score     0.020581
dtype: float64 

(100, 1) 
 mean_test_score    0.703398
std_test_score     0.020050
dtype: float64 

(100, 5) 
 mean_test_score    0.688991
std_test_score     0.020761
dtype: float64 

(100, 10) 
 mean_test_score    0.666000
std_test_score     0.020581
dtype: float64 



In [3]:
#### THE FINAL CHOICE IS: max_depth': 50, 'min_samples_leaf': 1

rs = 516

X_train = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_b/final_X_train_val.csv')
y_train = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_b/y_train_val.csv')

regression_forest = RandomForestRegressor(max_depth=50,
                                          min_samples_leaf=1,
                                          n_jobs=-1, 
                                          random_state=rs
                                          )


regression_forest.fit(X_train, y_train)

with open('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_b/trained_models/final_regression.pkl', 'wb') as f:
    pickle.dump(regression_forest, f)

/nfs/home/genovese/.venv/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [4]:
with open('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_b/trained_models/final_regression.pkl', 'rb') as f:
    model = pickle.load(f)

In [6]:
y_train_pred = model.predict(X_train).round(0)
print(r2_score(y_train_pred, y_train['target']))
print(accuracy_score(pd.Series(y_train_pred), pd.Series(y_train['target'])))

X_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_b/final_X_test.csv')
y_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_b/y_test.csv')

y_test_pred = model.predict(X_test).round(0)
print(r2_score(y_test_pred, y_test['target']))
print(accuracy_score(pd.Series(y_test_pred), pd.Series(y_test['target'])))

0.9391398463774282
0.610840108401084
0.4665866728985729
0.2590546347452425


In [7]:
def predict_dispersion(crosstab):
    total = 0
    for i, row in enumerate(crosstab.to_numpy()):
        for j, val in enumerate(row):
            total += val * abs(j - i)
    return total/(crosstab.sum().sum())*(crosstab.shape[1])

In [8]:
train_mat= pd.crosstab(y_train.target, y_train_pred)
test_mat = pd.crosstab(y_test.target, y_test_pred)

In [12]:
print(predict_dispersion(train_mat))
print(predict_dispersion(test_mat))

4.607479674796748
14.430325352977288


---

In [13]:
res  =pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/outputs/sample_b/classification_fine_tuning.csv', index_col=0)
for params, df in res.groupby(['param_max_depth', 'param_min_samples_leaf']):
    print(params, '\n', df[['mean_test_score', 'std_test_score']].mean(), '\n')

(6, 1) 
 mean_test_score    0.303414
std_test_score     0.016845
dtype: float64 

(6, 5) 
 mean_test_score    0.303413
std_test_score     0.011998
dtype: float64 

(6, 10) 
 mean_test_score    0.296910
std_test_score     0.016626
dtype: float64 

(50, 1) 
 mean_test_score    0.451161
std_test_score     0.015057
dtype: float64 

(50, 5) 
 mean_test_score    0.419181
std_test_score     0.021972
dtype: float64 

(50, 10) 
 mean_test_score    0.393710
std_test_score     0.017604
dtype: float64 

(100, 1) 
 mean_test_score    0.451161
std_test_score     0.015057
dtype: float64 

(100, 5) 
 mean_test_score    0.419181
std_test_score     0.021972
dtype: float64 

(100, 10) 
 mean_test_score    0.393710
std_test_score     0.017604
dtype: float64 



In [14]:
#### THE FINAL CHOICE IS: max_depth': 50, 'min_samples_leaf': 1

rs = 516

X_train = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_b/final_X_train_val.csv')
y_train = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_b/y_train_val.csv')

classification_forest = RandomForestClassifier(max_depth=50,
                                          min_samples_leaf=1,
                                          n_jobs=-1, 
                                          random_state=rs
                                          )


classification_forest.fit(X_train, y_train)

with open('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_b/trained_models/final_classification.pkl', 'wb') as f:
    pickle.dump(classification_forest, f)

/nfs/home/genovese/.venv/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [15]:
with open('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_b/trained_models/final_classification.pkl', 'rb') as f:
    model = pickle.load(f)

In [16]:
y_train_pred = model.predict(X_train).round(0)
print(r2_score(y_train_pred, y_train['target']))
print(accuracy_score(pd.Series(y_train_pred), pd.Series(y_train['target'])))

X_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_b/final_X_test.csv')
y_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_b/y_test.csv')

y_test_pred = model.predict(X_test).round(0)
print(r2_score(y_test_pred, y_test['target']))
print(accuracy_score(pd.Series(y_test_pred), pd.Series(y_test['target'])))

0.9989281618695413
0.9959891598915989
0.6832560065232645
0.46838551258440764


In [17]:
train_mat= pd.crosstab(y_train.target, y_train_pred)
test_mat = pd.crosstab(y_test.target, y_test_pred)

In [21]:
print(predict_dispersion(train_mat))
print(predict_dispersion(test_mat))

0.06319783197831978
11.972375690607734


---